# SWD low-level interaction example

This notebook will show you how to interact with a target over SWD using Hydrabus

In [ ]:
import pyHydrabus

## Initialization

First instanciate the SWD interface, then perform the SWD initialization sequence.
This module provides helpers to query the DP or any AP

In [ ]:
s = pyHydrabus.SWD()
s.bus_init()

## DP initialization

Start by reading the DPIDR

In [ ]:
print(f"DPIDR: {hex(s.read_dp(0))}")

## Debug domain initialization

In order to allow the MEM-AP to access the memory, the debug domain has to be powered on.
This is done by setting the `CDBGPWRUPREQ` bit in the `CTRL-STAT` DP register

In [ ]:
#Power up debug domain
s.write_dp(4, 0x50000000)

## SWD bus discovery

Once the DP is initialized, it is possible to scan the SWD bus and discover existing APs.
To perform the scan, simply query all AP addresses for the `IDR` register and display all results

In [ ]:
#Scan the SWD bus
for i in range(1):
    print(f"AP {i} IDCODE: {hex(s.read_ap(i, 0xfc))}")

In [ ]:
AP_ADDRESS = 0

## MEM-AP initialization

Now that we discovered the MEM-AP, we need to initialize it by setting various parameters in the `CSW` register at address 0)
 - Size[2:0] set to 32-bit transfers (`0b010`)
 - DeviceEn[6] bit set to enable

In [ ]:
CSW = s.read_ap(0, 0)
CSW |= 0b010
CSW |= 1<<6
s.write_ap(AP_ADDRESS,0,CSW)

## Memory space interaction

Now that the MEM-AP is initialized, it is possible to query any address in the memory space. This is done with the following sequence :
 - Set the `TAR` register at address 0x4 with the address to read or write
 - Read or write the `DRW` register at address 0xc to perform the read or write

In [ ]:
def read_cpu_address(address):
        s.write_ap(AP_ADDRESS, 0x4, address)
        return s.read_ap(0, 0xc)

def write_cpu_address(address, data):
    s.write_ap(AP_ADDRESS, 0x4, address)
    s.write_ap(AP_ADDRESS, 0xc, data)

## CPU control

Before we can read the flash memory, we need to make sure that the CPU does not access the flash at the same time as the debugger.
In order to avoid that, we can stop the CPU execution by writing to the `DHCSR` ARM register at address `0xE000EDF0` and set bits `C_HALT[1]` and `C_DEBUGEN[0]`.

To restart execution, disable the same bits in the DHCSR register

In [ ]:
def halt_cpu():
    #Write to MEM-AP DRW, 
    write_cpu_address(0xE000EDF0, 0xA05F0003)
 
def run_cpu():
    write_cpu_address(0xE000EDF0, 0xA05F0000)

In [ ]:
halt_cpu()

## Flash dump

Now that the code is disabled, we can start the dumping process. We need to specify the address to read from and the total number of bytes to read.

In [ ]:
#Get those information from the memory map
FLASH_BASE_ADDRESS = 0x08000000
FLASH_SIZE = 0x2000

buff = b''

for i in range(0, FLASH_SIZE,4):
    val = read_cpu_address(FLASH_BASE_ADDRESS+i).to_bytes(4, byteorder="little")
    buff = buff+val

In [ ]:
print(buff.hex())

In [ ]:
out = open('firmware.bin', 'wb')
out.write(buff)
out.close()